## 一、导入函数工具包

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow import keras
from sklearn.metrics import mean_absolute_error

## 二、导入数据

In [2]:
data_train = pd.read_csv('used_car_train_20200313.csv',sep = ' ')
data_test = pd.read_csv('used_car_testA_20200313.csv',sep = ' ')

## 三、数据清洗

In [3]:
data_train.replace(to_replace = '-', value = np.nan, inplace = True)
data_test.replace(to_replace = '-', value = np.nan, inplace = True)

In [4]:
data_train.fillna(data_train.median(),inplace= True)
data_test.fillna(data_train.median(),inplace= True)

In [5]:
#特征标签
tags = ['model','brand','bodyType','fuelType','regionCode','regionCode','regDate','creatDate','kilometer','notRepairedDamage','power','v_0', 'v_1', 'v_2', 'v_3', 'v_4', 'v_5', 'v_6',
       'v_7', 'v_8', 'v_9', 'v_10', 'v_11', 'v_12', 'v_13', 'v_14']

In [6]:
#修改异常数据
data_train['power'][data_train['power']>600]=600
data_test['power'][data_test['power']>600]=600

D:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
D:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
#特征归一化
min_max_scaler = MinMaxScaler()
min_max_scaler.fit(data_train[tags].values)
x = min_max_scaler.transform(data_train[tags].values)
x_ = min_max_scaler.transform(data_test[tags].values)

In [8]:
#获得y值
y = data_train['price'].values

In [9]:
#切分数据集
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2)

## 四、建模训练预测

In [10]:
model = keras.Sequential([
        keras.layers.Dense(250,activation='relu',input_shape=[26]), 
        keras.layers.Dense(250,activation='relu'), 
        keras.layers.Dense(250,activation='relu'), 
        keras.layers.Dense(1)])
model.compile(loss='mean_absolute_error',
                optimizer='Adam')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [11]:
model.fit(x_train,y_train,batch_size = 2048,epochs=50)

Train on 120000 samples
Epoch 1/50
120000/120000 [==============================] - 2s 16us/sample - loss: 5563.9603
Epoch 2/50
120000/120000 [==============================] - 1s 6us/sample - loss: 4267.2074
Epoch 3/50
120000/120000 [==============================] - 1s 9us/sample - loss: 3647.6043
Epoch 4/50
120000/120000 [==============================] - 1s 9us/sample - loss: 1971.3189
Epoch 5/50
120000/120000 [==============================] - 1s 6us/sample - loss: 1305.9669
Epoch 6/50
120000/120000 [==============================] - 1s 7us/sample - loss: 1150.1801
Epoch 7/50
120000/120000 [==============================] - 1s 6us/sample - loss: 1066.3974
Epoch 8/50
120000/120000 [==============================] - 1s 7us/sample - loss: 1004.5189
Epoch 9/50
120000/120000 [==============================] - 1s 6us/sample - loss: 957.2493
Epoch 10/50
120000/120000 [==============================] - 1s 6us/sample - loss: 920.7184
Epoch 11/50
120000/120000 [=============================

In [12]:
#比较训练集和测试集效果
mean_absolute_error(y_train,model.predict(x_train))

568.8469346729279

In [13]:
mean_absolute_error(y_test,model.predict(x_test))

578.0792317927043

In [14]:
#输出结果预测
y_=model.predict(x_)

In [15]:
data_test_price = pd.DataFrame(y_,columns = ['price'])
results = pd.concat([data_test['SaleID'],data_test_price],axis = 1)
results.to_csv('results.csv',sep = ',',index = None)